In [124]:
import numpy as np
from IPython.display import HTML, display
from pydrake.all import (
    AbstractValue, 
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    PiecewisePose,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    TrajectorySource,
    AddMultibodyPlantSceneGraph,
    Parser,
)
import math

In [2]:
from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad

In [3]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [134]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )
        
    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))

In [5]:
def MakeJacoStation(
    model_directives=None,
    filename=None,
    time_step=0.002,
):
    """
    Creates a kinova jaco station system, which is a sub-diagram containing:
     - A MultibodyPlant with populated via the Parser from the
       `model_directives` argument AND the `filename` argument.
     - A SceneGraph
     - For each model instance starting with ``
     
    Args:
        model_directives: a string containing any model directives to be parsed
        
        filename: a string containing the name of an sdf, urdf, mujoco xml. or
        model directives yaml file.
        
        time_step: the standard MultibodyPlant time step.

    """
    builder = DiagramBuilder()
    
    # add kinova jaco arm
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=time_step
    )
    parser = Parser(plant)
    if model_directives:
        parser.AddModelsFromString(model_directives, ".dmd.yaml")
    if filename:
        parser.AddModelsFromUrl(filename)
    plant.Finalize()
    
    # Export "cheat" ports.
    builder.ExportOutput(scene_graph.get_query_output_port(), "query_object")
    
    diagram = builder.Build()
    diagram.set_name("JacoStation")
    return diagram

In [110]:
class Jaco_Painter:
    def __init__(self, traj=None):
        builder = DiagramBuilder()
        # set up the systems of jaco station
        self.station = MakeJacoStation(
            filename="package://drake/manipulation/models/jaco_description/urdf/j2s7s300_sphere_collision.urdf")
        
        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")
        """
        controller_plant = self.station.GetSubsystemByName(
            "jaco_controller"
        ).get_multibody_plant_for_control()
        """
        
        # add trajectory source
        if traj is not None:
            traj_source = builder.AddSystem(PoseTrajectorySource(traj))
            # TODO:
            # add controller
        
        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder,
            self.station.GetOutputPort("query_object"),
            meshcat,
            params
        )
        
        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName("j2s7s300_end_effector")
        self.world_frame = self.plant.world_frame()
        
        context = self.CreateDefaultContext()
        self.diagram.ForcedPublish(context)
        
    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )
        
    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context
        )
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context
        )
        
        # provide initial states
        q0 = np.array(
            [
                1.8,
                3.44,
                3.14,
                0.76,
                4.63,
                4.49,
                5.03,
            ]
        )
        # get q_size from MbP
        num_positions = self.plant.num_positions()
        print("nq: {}".format(num_positions))
        num_velocities = self.plant.num_velocities()
        print("nq: {}".format(num_velocities))
        q0_ = np.zeros(num_positions)
        joint_index = 0
        q0_[0] = 0.
        q0_[1] = 0.
        q0_[2] = 0.
        q0_[3] = 1.
        #q0_[4] base x
        #q0_[5] base y
        #q0_[6] base z
        q0_[7] = 1.8
        q0_[8] = 3.44
        q0_[9] = 3.14
        q0_[10] = 0.76
        q0_[11] = 4.63
        q0_[12] = 4.49
        q0_[13] = 5.03
        """
        for i in q0:
            q0_[joint_index] = i
            joint_index += 1
        """
        #q0_[16] = 1.0
        print(q0_)
        # print(q_zero[:7])
        # qdot0 = np.zeros(num_positions)
        # set the joint positions of the jaco arm
        jaco = self.plant.GetModelInstanceByName("j2s7s300")
        self.plant.SetPositions(plant_context, jaco, q0_)
        self.plant.SetVelocities(plant_context, jaco, np.zeros(num_velocities))
        # TODO:
        return context
    
    def get_X_WG(self, context=None):
        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
            plant_context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        return X_WG
        
    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(1.0)
        
        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)

In [111]:
painter = Jaco_Painter()

nq: 17
nq: 16
[0.   0.   0.   1.   0.   0.   0.   1.8  3.44 3.14 0.76 4.63 4.49 5.03
 0.   0.   0.  ]


In [115]:
X_WG = painter.get_X_WG()
painter.visualize_frame("gripper_current", X_WG)

nq: 17
nq: 16
[0.   0.   0.   1.   0.   0.   0.   1.8  3.44 3.14 0.76 4.63 4.49 5.03
 0.   0.   0.  ]


In [114]:
radius = 0.1
p0 = [0.45, 0.0, 0.4]
R0 = RotationMatrix(np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]).T)
X_WorldCenter = RigidTransform(R0, p0)

num_key_frames = 10
thetas = np.linspace(0, 2 * np.pi, num_key_frames)

In [127]:
def compose_circular_key_frames(thetas, X_WorldCenter, X_WorldGripper_init, radius):
    """
    returns: a list of RigidTransforms
    """
    key_frame_poses_in_world = [X_WorldGripper_init]
    for theta in thetas:
        p_WorldCenter = X_WorldCenter.translation()
        # calculate each coordinates
        # TODO:
        # change radius depending on theta
        p_i = [radius + radius * math.cos(theta), 
               radius + radius * math.sin(theta),
               p_WorldCenter[2]
              ]
        this_pose = RigidTransform(RotationMatrix().MakeZRotation(theta), p_i)
        # TODO:
        key_frame_poses_in_world.append(this_pose)
        
    return key_frame_poses_in_world

In [128]:
def visualize_key_frames(frame_poses):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame("frame_{}".format(i), pose, length=0.05)
        
key_frame_poses = compose_circular_key_frames(
    thetas, X_WorldCenter, painter.get_X_WG(), radius
)
visualize_key_frames(key_frame_poses)

nq: 17
nq: 16
[0.   0.   0.   1.   0.   0.   0.   1.8  3.44 3.14 0.76 4.63 4.49 5.03
 0.   0.   0.  ]


In [129]:
X_WorldGripper_init = painter.get_X_WG()
total_time = 20
key_frame_poses = compose_circular_key_frames(
    thetas, X_WorldCenter, X_WorldGripper_init, radius
)
times = np.linspace(0, total_time, num_key_frames + 1)
traj = PiecewisePose.MakeLinear(times, key_frame_poses)

nq: 17
nq: 16
[0.   0.   0.   1.   0.   0.   0.   1.8  3.44 3.14 0.76 4.63 4.49 5.03
 0.   0.   0.  ]


In [135]:
painter = Jaco_Painter(traj)

nq: 17
nq: 16
[0.   0.   0.   1.   0.   0.   0.   1.8  3.44 3.14 0.76 4.63 4.49 5.03
 0.   0.   0.  ]


In [136]:
painter.paint(sim_duration=total_time)

nq: 17
nq: 16
[0.   0.   0.   1.   0.   0.   0.   1.8  3.44 3.14 0.76 4.63 4.49 5.03
 0.   0.   0.  ]


RuntimeError: Actuation input port for model instance j2s7s300 must be connected.